In [ ]:
#in this file I'll

# there'll be folders on the local computer /videoData/YYYY-MM/deviceName_YYYY-MM-DD

# in 10 years there'd be a max of 10 * 12 * 5 * 30 * 480 * 2 
# which is about 17 million files which does seem like a lot to check a lot
# however within the first year for me it'll probs be more like
# 12 * 2 * 30 * 480 * 2 = which is still 691k files damn

# we'll first be adding the timestamps to a parquet for each device with columns
# index, timestamp
# the device name we can get from the name of the df
# the location of the frame we can also look up
# the location of the frame in the video we can also calculate based on the video name

# once a parquet has been added we can delete it in the video store
# honestly we can use the existance of a parquet to tell if a folder has been logged




# the cameras will be moving their files into the folder

In [ ]:
# there'll be a direcory called videoMetaData/
# there'll be folders for each device?
# then there'll be folders for each kind of processing
#   time indexes, and why not, lets also put in file name and frame index
#   just poses for now
    # in the format time index, pose list object

# also for the cameras lets build them up one use case at a time
# general tracking multiclass classification
#   sitting - bedroom
#   standing
#   lying down
#   showering - bathroom
#   pooping
#   taking a shower
#   brushing teeth
#   putting corizone on
#   making food - kitchen
#   wiping nose - all

# later I'll have a different set of cameras to help with facial analysis
# I'll still have one recording I can test some processing strategies
# at linux computer

# 